In [2]:
import numpy as np
import cv2
from tqdm import tqdm
import os
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import time

In [3]:
train_dir = 'pictures'

In [4]:
def label_img(img):
    img_label = img.split('.')[0]
    return img_label

In [5]:
def create_train_data():
    imgs = []
    labels = []
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img)
        path = os.path.join(train_dir,img)
        img = cv2.imread(path)
        imgs.append([np.array(img)])
        labels.append(label)
    return imgs,labels

In [6]:
def apply_pca(x):
    pca = PCA(20)
    pca_x = [None] * len(x)
    for i in tqdm(range(len(x))):
        #apply pca
        img = x[i]
        blue= np.squeeze(img[:,:,0])
        green = np.squeeze(img[:,:,1])
        red = np.squeeze(img[:,:,2])
        red_transformed = pca.fit_transform(red)
        #Applying to Green channel and then applying inverse transform to transformed array.
        green_transformed = pca.fit_transform(green)
        #Applying to Blue channel and then applying inverse transform to transformed array.
        blue_transformed = pca.fit_transform(blue)
        img = np.stack((red_transformed, green_transformed, blue_transformed), axis = 2).tolist()
        pca_x[i] = (img)
    return pca_x

In [7]:
x, y = create_train_data()
x = np.squeeze(x)

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [00:40<00:00, 492.02it/s]


In [8]:
x = apply_pca(x)

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [18:02<00:00, 18.48it/s]


In [46]:
country_df = pd.DataFrame({'Country': y})
labelencoder = LabelEncoder()
country_df['Country'] = labelencoder.fit_transform(country_df['Country'])
country_df

,Country
0,0
1,0
2,0
3,0
4,0
...,...
19995,9
19996,9
19997,9
19998,9


In [12]:
df = pd.DataFrame({"X":x})
df = df.join(enc_df)

In [14]:
df.head()

Index(['X', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='object')

In [15]:
df.to_csv('df.csv.gzip', compression='gzip')  

In [17]:
df = pd.read_csv('df.csv.gzip', compression = 'gzip')

,Unnamed: 0,X,0,1,2,3,4,5,6,7,8,9
0,0,"[[[-515.656611475906, -123.66000314054135, -61...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"[[[1657.6059213222636, -506.26637486959527, 98...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,"[[[1699.1243830042176, -1195.6187278411367, 81...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,"[[[-242.87804479482452, 117.56884854261922, -7...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,"[[[472.3682836123981, -418.2295019853203, -143...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df = df.drop(df.columns[[0]],axis = 1)

In [67]:
class MyDataset():
 
    def __init__(self,x,y):
        
        self.x_train=torch.tensor(x,dtype=torch.float32)
        self.y_train=torch.tensor(y,dtype=torch.int64)
        self.y_train = torch.nn.functional.one_hot(self.y_train, num_classes= 10) 
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]


In [71]:
y = country_df["Country"]
md = MyDataset(x,y)

In [72]:
len(md)

20000

In [73]:
train_loader=DataLoader(
    md,
    batch_size=64, shuffle=True
)

In [74]:
class CountryCNN:
    def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
            self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
            self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)
            self.fc1 = nn.Linear(in_features= 64 * 6 * 6, out_features=500)
            self.fc2 = nn.Linear(in_features=500, out_features=50)
            self.fc3 = nn.Linear(in_features=50, out_features=2)
                   
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)
        
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)
        
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)
        
#         print(X.shape)
        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        
#         X = torch.sigmoid(X)
        return X

In [ ]:
model = CountryCNN().cuda()
losses = []
accuracies = []
epoches = 5
start = time.time()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
for epoch in range(epoches):
    epoch_loss = 0
    epoch_accuracy = 0
    for X, y in train_loader:
        X = X.cuda()
        y = y.cuda()
        preds = model(X)
        loss = loss_fn(preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        accuracy = ((preds.argmax(dim=1) == y).float().mean())
        epoch_accuracy += accuracy
        epoch_loss += loss
        print('.', end='', flush=True)
        
    epoch_accuracy = epoch_accuracy/len(train_dl)
    accuracies.append(epoch_accuracy)
    epoch_loss = epoch_loss / len(train_dl)
    losses.append(epoch_loss)
    print("Epoch: {}, train loss: {:.4f}, train accracy: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))


    with torch.no_grad():
        val_epoch_loss = 0
        val_epoch_accuracy = 0
        for val_X, val_y in valid_dl:
            val_X = val_X.cuda()
            val_y = val_y.cuda()
            val_preds = model(val_X)
            val_loss = loss_fn(val_preds, val_y)

            val_epoch_loss += val_loss            
            val_accuracy = ((val_preds.argmax(dim=1) == val_y).float().mean())
            val_epoch_accuracy += val_accuracy
        val_epoch_accuracy = val_epoch_accuracy/len(valid_dl)
        val_epoch_loss = val_epoch_loss / len(valid_dl)
        print("Epoch: {}, valid loss: {:.4f}, valid accracy: {:.4f}, time: {}\n".format(epoch, val_epoch_loss, val_epoch_accuracy, time.time() - start))